In [ ]:
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name blast blast`)

In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "StatsPlots"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
total_mem = 64
threads = max(1, Int(total_mem//8))
mem_per_cpu = Int(total_mem//threads)

In [ ]:
blast_task = "megablast"

In [ ]:
db = "nt_viruses"

In [ ]:
blastdbs_dir = mkpath(joinpath(homedir(), "workspace", "blastdbs"))

In [ ]:
blast_db = joinpath(blastdbs_dir, db)

In [ ]:
# conda run -n blast update_blastdb.pl --showall

In [ ]:
# Mycelia.download_blast_db(db=db, outdir=blastdbs_dir, source="ncbi", conda_env="blast")

In [ ]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

run_ids = sort(joint_metadata[!, "Run"])

sample_paths = joinpath.(data_dir, "SRA", run_ids)

In [ ]:
# sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
# sample_paths = filter(x -> isfile(joinpath.(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [ ]:
for sample_path in sample_paths[1:end]
    sample = basename(sample_path)
    blastn_directory = mkpath(joinpath(sample_path, "blastn"))
    assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    target_outfile = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    # # --mem=$(memory)G
    # if isfile(target_outfile)
    #         # @show sample, filesize(target_outfile)
    #     push!(file_sizes, filesize(target_outfile))
    # else
    #     @show sample
    # end

    cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db $(blast_db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
    submission = 
    `sbatch 
    --mail-user=cameron.prybol@gmail.com
    --mail-type=ALL
    --error=%x-%j.err
    --output=%x-%j.out
    --partition=batch
    --account=mpsnyder
    --nodes=1
    --ntasks=1
    --time=1-00:00:00    
    --cpus-per-task=$(threads)
    --mem-per-cpu=$(mem_per_cpu)G
    --job-name=blastn
    --wrap $cmd
    `
    run(submission)
end

In [ ]:
fasta_paths = filter(x -> !occursin(".ipynb_checkpoints", x) && occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "joint-reads"), join=true))
# sample_paths = filter(x -> isfile(joinpath.(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [ ]:
for fasta_path in fasta_paths
    # sample = basename(fasta_path)
    blastn_directory = dirname(fasta_path)
    assembled_fasta = fasta_path
    target_outfile = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    # # --mem=$(memory)G
    # if isfile(target_outfile)
    #         # @show sample, filesize(target_outfile)
    #     push!(file_sizes, filesize(target_outfile))
    # else
    #     @show sample
    # end

    cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db $(blast_db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
    submission = 
    `sbatch 
    --mail-user=cameron.prybol@gmail.com
    --mail-type=ALL
    --error=%x-%j.err
    --output=%x-%j.out
    --partition=batch
    --account=mpsnyder
    --nodes=1
    --ntasks=1
    --time=1-00:00:00    
    --cpus-per-task=$(threads)
    --mem-per-cpu=$(mem_per_cpu)G
    --job-name=blastn
    --wrap $cmd
    `
    run(submission)
end